In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import absolute, mean, std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
from sklearn import preprocessing, metrics
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import files
data = files.upload() 
df = pd.read_csv('df.csv')

Saving df.csv to df.csv


In [3]:
len(df)

301

In [4]:
df.head()

,Résident,Chambre,Etage,GIR,Protocole,Day Type,Semaine de l'année,Durée en Minutes,Mois,Année,Trimestre,Jour
0,3,70,1er,1,Entretien courant Avancé,Weekday,8,4.850933,2,2022,Trim 1,17
1,3,70,1er,1,Entretien courant Avancé,Weekday,3,9.375583,1,2022,Trim 1,12
2,3,70,1er,1,Entretien courant Avancé,Weekday,10,15.396083,3,2022,Trim 1,2
3,11,59,3ème,1,Entretien courant Avancé,Weekday,21,5.108050,5,2022,Trim 2,16
4,11,59,3ème,1,Entretien courant Avancé,Weekday,34,8.391583,8,2022,Trim 3,17


In [5]:
first_column = df.pop('Durée en Minutes')
df.insert(0, 'Durée en Minutes', first_column)

In [6]:
df.head()

,Durée en Minutes,Résident,Chambre,Etage,GIR,Protocole,Day Type,Semaine de l'année,Mois,Année,Trimestre,Jour
0,4.850933,3,70,1er,1,Entretien courant Avancé,Weekday,8,2,2022,Trim 1,17
1,9.375583,3,70,1er,1,Entretien courant Avancé,Weekday,3,1,2022,Trim 1,12
2,15.396083,3,70,1er,1,Entretien courant Avancé,Weekday,10,3,2022,Trim 1,2
3,5.108050,11,59,3ème,1,Entretien courant Avancé,Weekday,21,5,2022,Trim 2,16
4,8.391583,11,59,3ème,1,Entretien courant Avancé,Weekday,34,8,2022,Trim 3,17


In [7]:
label_encoder = preprocessing.LabelEncoder()
df['Etage']= label_encoder.fit_transform(df['Etage'])
df['Protocole']= label_encoder.fit_transform(df['Protocole'])
df['Day Type']= label_encoder.fit_transform(df['Day Type'])
df['Trimestre']= label_encoder.fit_transform(df['Trimestre'])
df.head()

,Durée en Minutes,Résident,Chambre,Etage,GIR,Protocole,Day Type,Semaine de l'année,Mois,Année,Trimestre,Jour
0,4.850933,3,70,1,1,1,0,8,2,2022,0,17
1,9.375583,3,70,1,1,1,0,3,1,2022,0,12
2,15.396083,3,70,1,1,1,0,10,3,2022,0,2
3,5.108050,11,59,3,1,1,0,21,5,2022,1,16
4,8.391583,11,59,3,1,1,0,34,8,2022,2,17


In [8]:
# X = df.iloc[:,3:]
X = df.loc[:,['GIR','Protocole']]
y = df.iloc[:,0]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
from sklearn.pipeline import make_pipeline

rid = make_pipeline(StandardScaler(with_mean=False), Ridge(100))
rid.fit(X_train, y_train)
# rid = Ridge(1.0, normalize=True).fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('ridge', Ridge(alpha=100))])

In [11]:
y_pred = rid.predict(X_test)

In [12]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [13]:
scores = cross_val_score(rid, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [14]:
scores = absolute(scores)
print('R2 score: {0:.2f}'.format(metrics.r2_score(y_test, y_pred)))
print('Mean MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

R2 score: 0.01
Mean MAE: 3.569 (0.502)
